In [1]:
import numpy as np
import pandas as pd

In [29]:
products = pd.read_csv('../data/amazon_baby_subset.csv')
important_words = list(pd.read_json('../data/important_words.json')[0])
print list(products)
print products.head(10)['name']

['name', 'review', 'rating', 'sentiment']
0    Stop Pacifier Sucking without tears with Thumb...
1      Nature's Lullabies Second Year Sticker Calendar
2      Nature's Lullabies Second Year Sticker Calendar
3                          Lamaze Peekaboo, I Love You
4    SoftPlay Peek-A-Boo Where's Elmo A Children's ...
5                            Our Baby Girl Memory Book
6    Hunnt&reg; Falling Flowers and Birds Kids Nurs...
7    Blessed By Pope Benedict XVI Divine Mercy Full...
8    Cloth Diaper Pins Stainless Steel Traditional ...
9    Cloth Diaper Pins Stainless Steel Traditional ...
Name: name, dtype: object


In [9]:
positive_reviews = len(products[products['sentiment'] > 0])
negative_review = len(products[products['sentiment'] < 0])


In [10]:
print 'Positive reviews : ',positive_reviews
print 'Negative reviews : ',negative_review

Positive reviews :  26579
Negative reviews :  26493


In [36]:
products = products.fillna({'review':''})
def remove_punctuation(word):
    import string
    return word.translate(None,string.punctuation)

In [40]:
products['review_clean'] = products['review'].apply(remove_punctuation)
products.head(5)

,name,review,rating,sentiment,review_clean
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...


In [41]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda x : x.split(' ').count(word))

In [49]:
products.head(5)

,name,review,rating,sentiment,review_clean,baby,one,great,love,use,...,seems,picture,completely,wish,buying,babies,won,tub,almost,either
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [52]:
contain_perfect = len(products[products['perfect'] > 0])
print 'Contains Perfect : ',contain_perfect

Contains Perfect :  2955


# Quiz 1)

## Contains Perfect : 2955

In [53]:
def get_numpy_data(data_frame,features,labels):
    data_frame['ones'] = 1
    features = ['ones'] + features
    feature_matrix = data_frame[features].as_matrix()
    label_matrix = data_frame[labels].as_matrix()
    return feature_matrix,label_matrix

### A function that returns numpy matrix for the dataframes

In [54]:
feature_matrix,sentiment = get_numpy_data(products,important_words,'sentiment')

In [56]:
print "Feature Matrix : ",feature_matrix.shape
print 'Sentiment Matrix : ',sentiment.shape

Feature Matrix :  (53072, 194)
Sentiment Matrix :  (53072,)


# Quiz 2)            

## 194 features in the feature_matrix('ones' plus the important words)

# Quiz 3)

## Number of features in the feature matrix is GREATER THAN OR THE SAME AS the number of features in the model

In [57]:
def predict_probability(feature_matrix,coefficients):
    score = np.dot(feature_matrix,coefficients)
    probability = 1/(1 + np.exp(-score))
    return probability

In [108]:
def feature_derivative(errors,feature):
    return np.dot(errors.transpose(),feature)

In [109]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

In [134]:
def logistic_regression(feature_matrix,sentiment,initial_coeff,step_size,max_iter):
    senti = sentiment
    sentiment = map(lambda y : 1 if y ==1 else 0,sentiment)
    for i in range(max_iter):
        prediction = predict_probability(feature_matrix,initial_coeff)
        error = sentiment - prediction
        grad = feature_derivative(error,feature_matrix)
        initial_coeff = initial_coeff + step_size*grad
        l = compute_log_likelihood(feature_matrix,senti,initial_coeff)
        if i%50 == 0:
            print 'Iterations : ',i,"Likelihood : ",l
    return initial_coeff

In [135]:
feature_matrix = feature_matrix
sentiment = sentiment
initial_coeff = np.array([0]*feature_matrix.shape[1])
step_size = 1e-7
max_iter = 301


In [136]:
W = logistic_regression(feature_matrix,sentiment,initial_coeff,step_size,max_iter)

Iterations :  0 Likelihood :  -36780.9176848
Iterations :  50 Likelihood :  -36498.933161
Iterations :  100 Likelihood :  -36230.4610235
Iterations :  150 Likelihood :  -35974.1285439
Iterations :  200 Likelihood :  -35728.8941877
Iterations :  250 Likelihood :  -35493.9225113
Iterations :  300 Likelihood :  -35268.5121268
Final Coefficients :  [  5.16220157e-03   1.55656966e-02  -8.50204675e-03   6.65460842e-02
   6.58907629e-02   5.01743882e-03  -5.38601484e-02  -3.50488413e-03
   6.47945868e-02   4.54356263e-02   3.98353364e-03   2.00775410e-02
   3.01350011e-02  -2.87115530e-02   1.52161964e-02   2.72592062e-04
   1.19448177e-02  -1.82461935e-02  -1.21706420e-02  -4.15110334e-02
   2.76820391e-03   1.77031999e-02  -4.39700067e-03   4.49764014e-02
   9.90916464e-03   8.99239081e-04  -1.36219516e-03   1.26859357e-02
   8.26466695e-03  -2.77426972e-02   6.10128809e-04   1.54084501e-02
  -1.32134753e-02  -3.00512492e-02   2.97399371e-02   1.84087080e-02
   2.86178752e-03  -1.05768015e-

# Quiz 4)

## After each iterations loglikelihood increases

In [139]:
products['score'] = np.dot(feature_matrix,W.transpose())
products['prediction'] = products['score'].apply(lambda z : 1 if z>0 else -1)

In [147]:
products.head(2)

,name,review,rating,sentiment,review_clean,baby,one,great,love,use,...,wish,buying,babies,won,tub,almost,either,ones,score,prediction
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0.051046,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,-0.029365,-1


In [141]:
predicted_positive = len(products[products['prediction'] == 1])
predicted_negative = len(products[products['prediction'] == -1])
print 'Predicted Positive : ',predicted_positive
print 'Predicted Negative : ',predicted_negative

Predicted Positive :  25126
Predicted Negative :  27946


# Quiz 5)

## Predicted Positive :  25126
## Predicted Negative :  27946

In [145]:
accurate = len(products[products['prediction'] == products['sentiment']])
accuracy = float(accurate)/len(products)
print 'Accurate Prediction : ',accurate
print 'Accuracy : ',accuracy

Accurate Prediction :  39903
Accuracy :  0.751865390413


# Quiz 6)

## Accurate Prediction :  39903
## Accuracy :  0.751865390413

In [148]:
coefficients = list(W[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)

In [150]:
print word_coefficient_tuples[:10]

[(u'great', 0.066546084170457709), (u'love', 0.065890762922123244), (u'easy', 0.064794586802578394), (u'little', 0.045435626308421372), (u'loves', 0.044976401394906038), (u'well', 0.030135001092107077), (u'perfect', 0.029739937104968459), (u'old', 0.020077541034775385), (u'nice', 0.018408707995268989), (u'daughter', 0.017703199905701697)]


In [152]:
print word_coefficient_tuples[-20:]

[(u'broke', -0.015537698955653887), (u'returned', -0.016001798500102516), (u'item', -0.01713786701085479), (u'buy', -0.017737543997218049), (u'time', -0.018246193486087036), (u'way', -0.018359460662945692), (u'tried', -0.018702371424325834), (u'could', -0.019846256660777207), (u'thought', -0.021394348543682488), (u'waste', -0.024042748071154956), (u'monitor', -0.024482100545891717), (u'return', -0.026592778462247283), (u'back', -0.027742697230661327), (u'get', -0.028711552980192585), (u'disappointed', -0.028978976142317068), (u'even', -0.030051249236035808), (u'work', -0.03306951529475273), (u'money', -0.038982037286487116), (u'product', -0.04151103339210889), (u'would', -0.053860148445203128)]
